In [22]:
import pandas as pd
import re

In [6]:
file_path = "C:/Users/Sumeet/Downloads/Data Engineering/Data Engineering/data - sample.xlsx"  
excel_data = pd.ExcelFile('C:/Users/Sumeet/Downloads/Data Engineering/Data Engineering/data - sample.xlsx')

In [8]:
attendance_df = excel_data.parse('Attendance_data')
student_df = excel_data.parse('Student_data')

In [10]:
attendance_df['attendance_date'] = pd.to_datetime(attendance_df['attendance_date'])
attendance_df = attendance_df.sort_values(by=['student_id', 'attendance_date'])

In [12]:
def find_absent_streaks(df, streak_length=3):
    results = []
    for student_id, group in df.groupby('student_id'):
        group = group.reset_index(drop=True)
        count = 0
        start_date = None
        for i, row in group.iterrows():
            if row['status'] == 'Absent':
                count += 1
                if count == 1:
                    start_date = row['attendance_date']
                if count > streak_length:
                    end_date = row['attendance_date']
            else:
                if count > streak_length:
                    results.append((student_id, start_date, group.loc[i-1, 'attendance_date'], count))
                count = 0
        if count > streak_length:
            results.append((student_id, start_date, group.iloc[-1]['attendance_date'], count))
    return results

In [14]:
streaks = find_absent_streaks(attendance_df)
result_df = pd.DataFrame(streaks, columns=['student_id', 'absence_start_date', 'absence_end_date', 'total_absent_days'])
result_df

,student_id,absence_start_date,absence_end_date,total_absent_days
0,107,2024-03-22,2024-03-25,4
1,108,2024-03-21,2024-11-05,5


In [26]:
import pandas as pd
import re

students = pd.DataFrame({
    'student_id': [101, 102, 103, 104, 105],
    'student_name': ['Alice Johnson', 'Bob Smith', 'Charlie Brown', 'David Lee', 'Eva White'],
    'parent_email': [
        'alice_parent@example.com',
        'bob_parent@example.com',
        'invalid_email.com',
        'invalid_email.com',
        'eva_white@example.com'
    ]
})

attendance = pd.DataFrame({
    'student_id': [101]*6 + [102]*6 + [103]*6 + [104]*6 + [105]*6,
    'date': pd.date_range(start='2024-03-01', periods=6).tolist() * 5,
    'status': ['Present', 'Absent', 'Absent', 'Absent', 'Absent', 'Present'] * 5
})

attendance['date'] = pd.to_datetime(attendance['date'])
attendance = attendance.sort_values(by=['student_id', 'date'])

def find_absent_streaks(data, limit=3):
    records = []
    for sid, group in data.groupby('student_id'):
        group = group.reset_index(drop=True)
        streak = 0
        start = None
        for i, row in group.iterrows():
            if row['status'] == 'Absent':
                streak += 1
                if streak == 1:
                    start = row['date']
            else:
                if streak > limit:
                    end = group.loc[i - 1, 'date']
                    records.append((sid, start, end, streak))
                streak = 0
        if streak > limit:
            end = group.iloc[-1]['date']
            records.append((sid, start, end, streak))
    return records

absent_info = find_absent_streaks(attendance)
absent_df = pd.DataFrame(absent_info, columns=[
    'student_id', 'absence_start_date', 'absence_end_date', 'total_absent_days'
])

data = pd.merge(absent_df, students, on='student_id')

def is_valid(email):
    return re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*@[a-zA-Z]+\.(com)$', email) is not None

data['email'] = data['parent_email'].apply(lambda x: x if is_valid(x) else None)

def generate_msg(row):
    if row['email']:
        start_date = row['absence_start_date'].strftime('%d-%m-%Y')
        end_date = row['absence_end_date'].strftime('%d-%m-%Y')
        return f"Dear Parent, your child {row['student_name']} was absent from {start_date} to {end_date} for {row['total_absent_days']} days. Please ensure their attendance improves."
    return None

data['msg'] = data.apply(generate_msg, axis=1)

data['absence_start_date'] = data['absence_start_date'].dt.strftime('%d-%m-%Y')
data['absence_end_date'] = data['absence_end_date'].dt.strftime('%d-%m-%Y')

final = data[['student_id', 'absence_start_date', 'absence_end_date', 'total_absent_days', 'email', 'msg']]
print(final)

   student_id absence_start_date absence_end_date  total_absent_days  \
0         101         02-03-2024       05-03-2024                  4   
1         102         02-03-2024       05-03-2024                  4   
2         103         02-03-2024       05-03-2024                  4   
3         104         02-03-2024       05-03-2024                  4   
4         105         02-03-2024       05-03-2024                  4   

                      email                                                msg  
0  alice_parent@example.com  Dear Parent, your child Alice Johnson was abse...  
1    bob_parent@example.com  Dear Parent, your child Bob Smith was absent f...  
2                      None                                               None  
3                      None                                               None  
4     eva_white@example.com  Dear Parent, your child Eva White was absent f...  
